<a href="https://colab.research.google.com/github/saisuresh1999/ML-DL-Technical-Interview-CICIDS17-/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from time import time
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


In [8]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn
import time

from numpy import array

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.svm import LinearSVC


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import completeness_score, homogeneity_score, v_measure_score

from sklearn.model_selection import train_test_split

In [2]:
datapath = "/content/drive/MyDrive/Dataset_clean.csv"
df=pd.read_csv(datapath,header=0,low_memory=False)


In [3]:
df['Label'] = df['Label'].apply(lambda x: 0 if 'BENIGN' in x else 1)


In [4]:
"""# Train Test Split
t0 = time()
X_train, X_test, y_train,y_test = train_test_split(df.iloc[:,0:-1], df['Label'], test_size=0.3, random_state=42)
tt = time()-t0
print ("Split dataset in {} seconds".format(round(tt,3)))
"""

Split dataset in 4.457 seconds


In [6]:
# xs=feature vectors, ys=labels
xs = df.iloc[:,0:-1]
ys = df['Label']

# split dataset 
x_train, x_temp, y_train, y_temp = train_test_split(xs, ys, test_size=0.4, random_state=42)
x_test, x_validate, y_test, y_validate = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [9]:
# Normalise
min_max_scaler = MinMaxScaler().fit(x_train)

# Apply normalisation to dataset
x_train = min_max_scaler.transform(x_train)
x_validate = min_max_scaler.transform(x_validate)
x_test = min_max_scaler.transform(x_test)

# All values between 0 and 1
pd.Series(x_train.flatten()).describe()

count    1.135240e+08
mean     9.302320e-02
std      2.607999e-01
min      0.000000e+00
25%      0.000000e+00
50%      2.449293e-07
75%      6.410256e-03
max      1.000000e+00
dtype: float64

In [11]:

features = SelectKBest(score_func=chi2, k=x_train.shape[1])

#fit features to the training dataset
fit = features.fit(x_train, y_train)

In [12]:
features = SelectKBest(score_func=chi2, k=40)
fit = features.fit(x_train, y_train)
x_train = fit.transform(x_train)
x_test = fit.transform(x_test)
x_validate = fit.transform(x_validate)

In [14]:
new_features = df.columns[features.get_support(indices=True)]


In [15]:
classifier = LinearSVC()


In [17]:
# fit the model
start = time.time()
classifier.fit(x_train, y_train)
end = time.time()
training_time = end - start

# predict validation
start = time.time()
y_predicted = classifier.predict(x_validate)
end = time.time()
predict_time = end  - start
print(training_time, predict_time)

confusion_svm_1 = pd.crosstab(y_validate, y_predicted)
confusion_svm_1

191.967120885849 0.06544375419616699


col_0,0,1
Label,,
0,399455,7756
1,21676,56259


In [18]:
precision, recall, fscore, support = score(y_validate, y_predicted)
d = {'attack': [0,1], 'precision': precision, 'recall' : recall, 'fscore': fscore}
results = pd.DataFrame(data=d)
results

,attack,precision,recall,fscore
0,0,0.948529,0.980953,0.964469
1,1,0.878841,0.721871,0.792659
